In [1]:
from utils.structure import *
from utils.predicter import *
from utils.solver import *

In [2]:
# build structure and display
test_datas = open('./data/test_demo.txt', 'r')
test_datas = test_datas.readlines()
for test_data in test_datas:
    s = Structure(test_data)
#     print(s.data)
#     print(s.display())

In [3]:
s = Structure(test_datas[11])
print(s.display())

solver = BasicSolver(s)
# solver.idxes_need_to_solve

+-----+-----+-----+
|. . 1|. 4 7|. . 2|
|     |     |     |
|. 7 4|. 3 2|9 . .|
|     |     |     |
|2 8 3|. 5 .|. . .|
+-----+-----+-----+
|4 5 9|3 2 6|7 8 1|
|     |     |     |
|8 1 2|. 7 .|. . 9|
|     |     |     |
|7 3 6|. . 8|2 4 5|
+-----+-----+-----+
|. . 7|. 8 .|1 2 3|
|     |     |     |
|. 2 5|. . .|. 9 .|
|     |     |     |
|. . 8|2 6 .|5 7 .|
+-----+-----+-----+



In [6]:
# solver.check_idx_only(62)
# print(solver.ready)
# solver.check_idx_last_left(62)
# print(solver.ready)
# print(solver.check_scanned_drop('9'))
print(solver.ready)
print(solver.check_group_drop('9'))
print(solver.ready)
solver.update()
print(solver.ready)

[]
Before grouped drop, scanned drop can also make some changes.
[0, 1, 3, 21, 23, 48, 49, 54, 55, 57, 59, 72, 73, 77]
+-----+-----+-----+
|    1|  4 7|    2|
|     |     |     |
|  7 4|  3 2|9    |
|     |     |     |
|2 8 3|  5  |     |
+-----+-----+-----+
|4 5 9|3 2 6|7 8 1|
|     |     |     |
|8 1 2|  7  |    9|
|     |     |     |
|7 3 6|  . 8|2 4 5|
+-----+-----+-----+
|    7|  8  |1 2 3|
|     |     |     |
|  2 5|     |  9  |
|     |     |     |
|    8|2 6  |5 7  |
+-----+-----+-----+

False
[(49, '9')]
[]


In [7]:
print(solver.display())

+-----+-----+-----+
|. . 1|. 4 7|. . 2|
|     |     |     |
|. 7 4|. 3 2|9 . .|
|     |     |     |
|2 8 3|. 5 .|. . .|
+-----+-----+-----+
|4 5 9|3 2 6|7 8 1|
|     |     |     |
|8 1 2|. 7 .|. . 9|
|     |     |     |
|7 3 6|. 9 8|2 4 5|
+-----+-----+-----+
|. . 7|. 8 .|1 2 3|
|     |     |     |
|. 2 5|. . .|. 9 .|
|     |     |     |
|. . 8|2 6 .|5 7 .|
+-----+-----+-----+



In [8]:
solver.steps

[(49, '9')]

In [9]:
tmp = "	 	 	1	?	4	7	 	 	2\
	×	7	4	×	3	2	9	×	×\
	2	8	3	?	5	?	 	 	 \
	4	5	9	3	2	6	7	8	1\
	8	1	2	×	7	×	 	 	9\
	7	3	6	×	 	8	2	4	5\
	 	 	7	?	8	?	1	2	3\
	×	2	5	×	×	×	×	9	×\
	 	 	8	2	6	?	5	7	 "

In [10]:
tmp = tmp.replace(' ', '.').replace('×', '.').replace('?','.').replace('	', '')
print(len(','.join(list(tmp))))
','.join(list(tmp))

161


'.,.,1,.,4,7,.,.,2,.,7,4,.,3,2,9,.,.,2,8,3,.,5,.,.,.,.,4,5,9,3,2,6,7,8,1,8,1,2,.,7,.,.,.,9,7,3,6,.,.,8,2,4,5,.,.,7,.,8,.,1,2,3,.,2,5,.,.,.,.,9,.,.,.,8,2,6,.,5,7,.'